In [1]:
from hgb_maternal_disorder_paf_generation import *
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
draw = 'draw_0'
location = 'pakistan'

In [4]:
results_directory = os.getcwd() + '/'

In [5]:
#get_simulated_population(location, draw)

In [7]:
calculate_paf_hemorrhage(location, draw)

2025-09-15 08:40:29.796 | INFO     | simulation_1-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model16.0/pakistan.hdf.
2025-09-15 08:40:29.799 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-15 08:40:29.800 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-15 08:40:29.799 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-15 08:40:29.800 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-15 08:41:04.951 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-15 08:41:04.953 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['st

age_group
10_to_14    0.359031
15_to_19    0.287729
20_to_24    0.245374
25_to_29    0.250555
30_to_34    0.245782
35_to_39    0.259342
40_to_44    0.268225
45_to_49    0.266679
other       0.306116
Name: hemoglobin_on_maternal_hemorrhage.relative_risk, dtype: float64

In [8]:
calculate_paf_sepsis(location, draw)

2025-09-15 08:41:43.035 | INFO     | simulation_2-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model16.0/pakistan.hdf.
2025-09-15 08:41:43.036 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-15 08:41:43.037 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-15 08:41:43.036 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-15 08:41:43.037 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.


2025-09-15 08:42:15.627 | WARNING  | simulation_2-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-15 08:42:15.629 | WARNING  | simulation_2-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-15 08:42:15.629 | WARNING  | simulation_2-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-15 08:42:16.027 | INFO     | simulation_2-results_manager:134 - The following stratifications are registered but not used by any observers: 
['sex']
2025-09-15 08:42:16.027 | INFO     | simulation_2-results_manager:134 - The following stratifications are registered but not used by any observers: 
['sex']


age_group
10_to_14    0.198284
15_to_19    0.161155
20_to_24    0.146736
25_to_29    0.147361
30_to_34    0.144888
35_to_39    0.150419
40_to_44    0.156080
45_to_49    0.155915
other       0.168187
Name: hemoglobin_on_maternal_sepsis_and_other_maternal_infections.relative_risk, dtype: float64

# Next steps
- Add depressive disorders to calculation
- Test different population sizes and plot PAFs 
- Do more testing to figure out if/why PAFs in artifact are so different than PAFs calculated in the interative sim here
- Run for all draws, locations, age groups, and outcomes and save new CSVs